# 使用自定义Image进行推理

需要用到Tensorflow 和Keras , kernel 选择conda_tensorflow_p36

## 把训练好的模型存放到指定路径下
`output/model/tf_server/`

目录结构如下
```
└── tf_server
    └── 1
        ├── saved_model.pb
        └── variables
            ├── variables.data-00000-of-00001
            └── variables.index
```

## Build容器
对外提供服务时，需要使用容器方式。  
图片分类本身可不使用自定义容器，这里只是演示如何自定义容器。

In [8]:
%%sh
docker build -t sagemaker-cat-vs-dog .

Sending build context to Docker daemon  434.2MB
Step 1/5 : FROM tensorflow/serving
 ---> 25820e475139
Step 2/5 : RUN apt-get update && apt-get install -y --no-install-recommends nginx git
 ---> Using cache
 ---> ab90e47298bb
Step 3/5 : COPY ./output/tf_server  /model
 ---> a2de54dc1c21
Step 4/5 : COPY nginx.conf /etc/nginx/nginx.conf
 ---> eba2a10b8590
Step 5/5 : ENTRYPOINT service nginx start | tensorflow_model_server --rest_api_port=8501  --model_name=sagemaker-demo  --model_base_path=/model
 ---> Running in 722795692ca9
Removing intermediate container 722795692ca9
 ---> 596182d56d14
Successfully built 596182d56d14
Successfully tagged sagemaker-cat-vs-dog:latest


## 本地运行docker

In [9]:
%%sh
docker run -p 8501:8501 -d sagemaker-cat-vs-dog

f8bebc4fce5867c4ee7d7b1e545909bed3331b921ae0c800a519074dbc00e86c


## 用本地Docker测试

In [10]:
import requests
import numpy as np
import json
import boto3 
IMAGE_WIDTH = 150 
IMAGE_HEIGHT = 150 

# 修改测试图片地址
image_paths = 'test/cat.681.jpg'
#image_paths = 'test/dog.592.jpg'
model_server_url = 'http://127.0.0.1:8501/v1/models/sagemaker-demo:predict'

In [11]:
from keras.preprocessing import image

images = image.load_img(image_paths, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
input_image = image.img_to_array(images)
input_image = np.expand_dims(input_image, axis=0)
input_image /= 255.

input_images = input_image.tolist()
data = json.dumps({"inputs":input_images})

headers = {"content-type": "application/json"}


json_response = requests.post(model_server_url, data=data, headers=headers)
json_result = json.loads(json_response.text)
print(json_result)

{'outputs': [[0.999354303, 0.00064571941]]}


In [12]:
# 根据需要修改class name，按字典序
class_name=['cat','dog']
class_name[np.argmax(json_result["outputs"])]

'cat'

## 推送docker镜像到ECR

In [13]:
%%sh
algorithm_name=sagemaker-cat-vs-dog
REGION=$(aws configure get region)
ACCOUNT=$(aws sts get-caller-identity --query Account --output text)

# If the ECS repository doesn't exist, creates it.
aws ecr create-repository --repository-name ${algorithm_name} > /dev/null 2>&1

# ECR requires the image name to be in this format:
REPOSITORY_NAME=${ACCOUNT}.dkr.ecr.${REGION}.amazonaws.com.cn/${algorithm_name}:latest

# Tags the image with the expect ECR format
docker tag ${algorithm_name} ${REPOSITORY_NAME}

# Allows docker access to ECR
$(aws ecr get-login --no-include-email)

# pushes the image to ECR
docker push ${REPOSITORY_NAME}

Login Succeeded
The push refers to repository [315505707008.dkr.ecr.cn-northwest-1.amazonaws.com.cn/sagemaker-cat-vs-dog]
c452c39d25a7: Preparing
e450e450b87a: Preparing
aa9f70b79e1c: Preparing
3392ec9e29bf: Preparing
66bdb0dbda05: Preparing
2b4a85ea7e13: Preparing
e41dc69483d3: Preparing
28ba7458d04b: Preparing
838a37a24627: Preparing
a6ebef4a95c3: Preparing
b7f7d2967507: Preparing
e41dc69483d3: Waiting
28ba7458d04b: Waiting
838a37a24627: Waiting
a6ebef4a95c3: Waiting
b7f7d2967507: Waiting
2b4a85ea7e13: Waiting
66bdb0dbda05: Layer already exists
aa9f70b79e1c: Layer already exists
3392ec9e29bf: Layer already exists
2b4a85ea7e13: Layer already exists
28ba7458d04b: Layer already exists
e41dc69483d3: Layer already exists
838a37a24627: Layer already exists
b7f7d2967507: Layer already exists
a6ebef4a95c3: Layer already exists
c452c39d25a7: Pushed
e450e450b87a: Pushed
latest: digest: sha256:529da078144f835127f4bb5b8bb38edef3a5ca599a25ee921862f97104e51162 size: 2621


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## 部署模型到SageMaker

In [14]:
from sagemaker.model import Model
import sagemaker
from sagemaker import get_execution_role

#role = get_execution_role()
role="arn:aws-cn:iam::315505707008:role/service-role/AmazonSageMaker-ExecutionRole-20200430T124235" 
image_uri = "315505707008.dkr.ecr.cn-northwest-1.amazonaws.com.cn/sagemaker-cat-vs-dog"
endpoint_name = "sagemaker-cat-vs-dog"
my_model = Model(
            role=role,
            image_uri=image_uri)

#该步骤大概需要10分钟
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.t2.medium'
                                )

-------------!

## 推理
### 读取数据

In [23]:
from keras.preprocessing import image
import json
import numpy as np

IMAGE_WIDTH = 150 
IMAGE_HEIGHT = 150 
# 修改测试图片地址
image_paths = 'test/cat.681.jpg'
#image_paths = 'test/dog.592.jpg'
images = image.load_img(image_paths, target_size=(IMAGE_WIDTH, IMAGE_HEIGHT))
input_image = image.img_to_array(images)
input_image = np.expand_dims(input_image, axis=0)
input_image /= 255.

input_images = input_image.tolist()

data = json.dumps({"name": 'tensorflow/serving/predict',"signature_name":'predict',"inputs":input_images})

In [24]:
input_image.shape

(1, 150, 150, 3)

### 方式一、使用boto3

In [16]:
import boto3
endpoint_name = "sagemaker-cat-vs-dog"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=data)
print(response)

{'ResponseMetadata': {'RequestId': 'ab1d44b7-7339-4b17-95c9-2e2bebc78c10', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ab1d44b7-7339-4b17-95c9-2e2bebc78c10', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 24 Aug 2020 09:57:30 GMT', 'content-type': 'application/json', 'content-length': '97'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7ff0fd6aaef0>}


In [17]:
response_body = response['Body'] 
body= response_body.read()
results = json.loads(body.decode('utf-8'))
print(results)

{'outputs': [[0.999354303, 0.00064571941]]}


### 方式二、使用SageMaker SDK

In [18]:
import sagemaker
xgb_predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session())

In [19]:
result = xgb_predictor.predict(data)
results = json.loads(result.decode('utf-8'))
print(results)

{'outputs': [[0.999354303, 0.00064571941]]}
